In [32]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
import random as rd
import itertools
import matplotlib.pyplot as plt
import heapq
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import MeanShift,estimate_bandwidth
from sklearn.cluster import DBSCAN   
from sklearn.neighbors import NearestNeighbors 
from sklearn.preprocessing import StandardScaler

%matplotlib inline
from sklearn.mixture import BayesianGaussianMixture,GaussianMixture
import scipy
from collections import Counter

import itertools
import time
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
import pyecharts

from allpairspy import AllPairs
import itertools
from sklearn.metrics import classification_report,accuracy_score

plt.rcParams['font.sans-serif'] = ['SimHei'] 
plt.rcParams['axes.unicode_minus'] = False  

# test ODD

In [33]:
ODD_FILE_PATH = './Testing_Results.csv'
odd = pd.read_csv(ODD_FILE_PATH)
odd

,vs,dif_speed,dis,result
0,1,0,5,0
1,1,0,6,0
2,1,0,7,0
3,1,0,8,0
4,1,0,9,0
...,...,...,...,...
68330,43,2,46,0
68331,43,2,47,0
68332,43,2,48,0
68333,43,2,49,0


In [34]:
FEATURE = ['vs','dif_speed','dis']
name = np.array(FEATURE)
X_qualified_All=odd[FEATURE].values
Y_all=odd['result'].values

# Calculate the PDF of each scenarios

In [35]:
DATA_FILE_PATH = './carfollowing_from_HighD.csv'
data = pd.read_csv(DATA_FILE_PATH)
data=data[['vs','dif_speed','dis']]
data

,vs,dif_speed,dis
0,35.02,0.80,22.67
1,36.62,1.71,25.39
2,24.53,-3.85,38.06
3,36.57,0.74,38.83
4,31.15,-0.16,35.29
...,...,...,...
30287,34.96,1.00,48.41
30288,31.91,-0.45,35.03
30289,32.76,-3.43,21.48
30290,32.84,-3.11,47.72


In [ ]:
import warnings
warnings.filterwarnings("ignore")

rdList = np.random.choice(data.shape[0],size=5000)

x_train = np.array(data)[rdList,:]
x_f = x_train

BEST_COMPONENTS = 0
BEST_BIC = 99999
for i in range(20):
    BGMM = GaussianMixture(n_components=i+1,tol=0.00001,max_iter=10000)
    BGMM.fit(x_f)
    bic = BGMM.bic(x_f)
    if bic < BEST_BIC:
        BEST_BIC = bic
        BEST_COMPONENTS = i+1
BGMM = GaussianMixture(n_components=BEST_COMPONENTS,tol=0.0001,max_iter=10000)
BGMM.fit(x_f) 
print("best_components:%d"%BEST_COMPONENTS)

In [38]:
def computeGMMpdf(BGMM, sample,scaler=None):
    if scaler is not None:
        sample = scaler.transform(sample)
    for i in range(BGMM.weights_.shape[0]):
        weights = BGMM.weights_[i] 
        mean = BGMM.means_[i, :]
        cov = BGMM.covariances_[i, :, :]
        mn = scipy.stats.multivariate_normal(mean=mean, cov=cov)
        if i == 0:
            res = mn.pdf(sample) * weights
        else:
            res += mn.pdf(sample) * weights 
    return res

In [ ]:
score = computeGMMpdf(BGMM,X_qualified_All)
p_x_all = score/score.sum() 

In [ ]:
score.sum()

In [ ]:
p_x_all

In [ ]:
x_p_frame = pd.DataFrame(X_qualified_All)
x_p_frame.columns = ['vs','dif_speed','dis']
x_p_frame['p'] = p_x_all
danger_rate=p_x_all[Y_all==1].sum()

# Evaluation Indicators

In [ ]:
evaluation_list = ['Samples','Methods','Input coverage','2-wise coverage','Number of collisions','Coverage of collisions','Collision rate estimation','Estimation Error'] 
evaluation_matrix = pd.DataFrame(columns=evaluation_list)

In [ ]:
def evaluation(time,name,sample,result,p_is=None,ment=False):
    array_tmp = [time,name]
    global evaluation_matrix,danger_rate,x_p_frame # 定义全局变量，然后会在函数外面寻找变量的定义，在函数内部对变量操作之后，依然是全局变量。

    # Input coverage
    ODD_cov=sample.shape[0]/68335
    array_tmp+=[ODD_cov]
    
    # 2-wise coverage
    pair_all_modify = 6566
    pair_found = np.unique(sample[:,0:2],axis=0).shape[0] + np.unique(sample[:,1:3],axis=0).shape[0] + np.unique(sample[:,[0,2]],axis=0).shape[0]
    pair_cov=pair_found/pair_all_modify
    array_tmp+=[pair_cov]
    
    # Number of collisions
    danger_find=result.sum()
    array_tmp+=[danger_find]
    
    # Coverage of collisions
    danger_cov=danger_find/6320
    array_tmp+=[danger_cov]
    
    # Collision rate estimation
    sample= pd.DataFrame(sample)
    sample['Y']=result
    sample.columns = ['vs','dif_speed','dis','Y']
    sample_with_rate=pd.merge(sample,x_p_frame,how='left')
    collision_rate=sample_with_rate[sample_with_rate.Y==1].p.sum()
    array_tmp+=[collision_rate]  
    
    
    # Estimation Error
    array_tmp += [np.abs(collision_rate-danger_rate)/danger_rate]

    
    #to evaluation_matrix
    evaluation_matrix.loc[evaluation_matrix.shape[0]] = array_tmp
    return array_tmp

# PSO

In [ ]:
from test_base_data import testBaseData
sut = testBaseData("Testing_Results.csv",name)

In [ ]:
from Grid_Search import grid
para_tuning = grid()
print(para_tuning.shape)

In [ ]:
%%time

k = 0

#from idm_pso import IDMDanger
#sut_danger=IDMDanger(name)

from pso import PSO
test_time=[1690,3000,4000,5000,6000,7000,8000,9000,10000,20000,30000,40000,50000,60000,68335]

from Grid_Search import grid
para_tuning = grid()
print(para_tuning.shape[0])
for i in range(para_tuning.shape[0]):
    para_1,para_2,para_3 = para_tuning[i][0],para_tuning[i][1],para_tuning[i][2]

    for NUM in test_time:
        SAMPLE_NUM = NUM
        pso = PSO(X_qualified_All,pN=int(SAMPLE_NUM*0.1),w=para_1,c1=para_2,c2=para_3)#int(SAMPLE_NUM*0.2)
        sample = pso.init_population(X_qualified_All,pN=int(SAMPLE_NUM*0.1))
        result = -sut.test(sample)
        history=sample
        for i in range(10):
            sample, fit = pso.train(sample,result)
            result = -sut.test(sample)
            history=np.concatenate((history,sample),axis=0)
        
        
        sample_use=np.unique(history,axis=0)
        result_use=sut.test(sample_use)
        result_use=np.nan_to_num(result_use, nan=0) #only count avoidable collisions
        
        temp_evaluate_result=evaluation(SAMPLE_NUM,'pso',sample_use,result_use)
        
        print(temp_evaluate_result)

In [ ]:
evaluation_matrix